In [22]:
%%bash
# SR from https://dodcio.defense.gov/Portals/0/Documents/CMMC/AssessmentGuideL2v2.pdf
jq -s '[.[]] | flatten' r2/*.json > r2.json

In [23]:
missing_req_titles = {
    "03.01.01": "Authorized Access Control",
    "03.01.02": "Transaction & Function Control",
    "03.01.03": "Control CUI Flow",
    "03.01.04": "Separation of Duties",
    "03.01.05": "Least Privilege",
    "03.01.06": "Non-Privileged Account Use",
    "03.01.07": "Privileged Functions",
    "03.01.08": "Unsuccessful Logon Attempts",
    "03.01.09": "Privacy & Security Notices",
    "03.01.10": "Session Lock",
    "03.01.11": "Session Termination",
    "03.01.12": "Control Remote Access",
    "03.01.13": "Remote Access Confidentiality",
    "03.01.14": "Remote Access Routing",
    "03.01.15": "Privileged Remote Access",

    "03.01.16": "Wireless Access Authorization",
    "03.01.17": "Wireless Access Protection",
    "03.01.18": "Mobile Device Connection",
    "03.01.19": "Encrypt CUI on Mobile",
    "03.01.20": "External Connections",
    "03.01.21": "Portable Storage Use",
    "03.01.22": "Control Public Information",

    "03.02.01": "Role-Based Risk Awareness",
    "03.02.02": "Role-Based Training",
    "03.02.03": "Insider Threat Awareness",

    "03.03.01": "System Auditing",
    "03.03.02": "User Accountability",
    "03.03.03": "Event Review",
    "03.03.04": "Audit Failure Alerting",
    "03.03.05": "Audit Correlation",
    "03.03.06": "Reduction & Reporting",
    "03.03.07": "Authoritative Time Source",
    "03.03.08": "Audit Protection",
    "03.03.09": "Audit Management",

    "03.04.01": "System Baselining",
    "03.04.02": "Security Configuration Enforcement",
    "03.04.03": "System Change Management",
    "03.04.04": "Security Impact Analysis",
    "03.04.05": "Access Restrictions for Change",
    "03.04.06": "Least Functionality",
    "03.04.07": "Nonessential Functionality",
    "03.04.08": "Application Execution Policy",
    "03.04.09": "User-Installed Software",

    "03.05.01": "Identification",
    "03.05.02": "Authentication",
    "03.05.03": "Multifactor Authentication",
    "03.05.04": "Replay-Resistant Authentication",
    "03.05.05": "Identifier Reuse",
    "03.05.06": "Identifier Handling",
    "03.05.07": "Password Complexity",
    "03.05.08": "Password Reuse",
    "03.05.09": "Temporary Passwords",
    "03.05.10": "Cryptographically-Protected Passwords",
    "03.05.11": "Obscure Feedback",

    "03.06.01": "Incident Handling",
    "03.06.02": "Incident Reporting",
    "03.06.03": "Incident Response Testing",

    "03.07.01": "Perform Maintenance",
    "03.07.02": "System Maintenance Control",
    "03.07.03": "Equipment Sanitization",
    "03.07.04": "Media Inspection",
    "03.07.05": "Nonlocal Maintenance",
    "03.07.06": "Maintenance Personnel",

    "03.08.01": "Media Protection",
    "03.08.02": "Media Access",
    "03.08.03": "Media Disposal",
    "03.08.04": "Media Markings",
    "03.08.05": "Media Accountability",
    "03.08.06": "Portable Storage Encryption",
    "03.08.07": "Removeable Media",
    "03.08.08": "Shared Media",
    "03.08.09": "Protect Backups",

    "03.09.01": "Screen Individuals",
    "03.09.02": "Personnel Actions",

    "03.10.01": "Limit Physical Access",
    "03.10.02": "Monitor Facility",
    "03.10.03": "Escort Visitors",
    "03.10.04": "Physical Access Logs",
    "03.10.05": "Manage Physical Access",
    "03.10.06": "Alternative Work Sites",

    "03.11.01": "Risk Assessments",
    "03.11.02": "Vulnerability Scan",
    "03.11.03": "Vulnerability Remediation",

    "03.12.01": "Security Control Assessment",
    "03.12.02": "operational Plan of Action",
    "03.12.03": "Security Control Monitoring",
    "03.12.04": "System Security Plan",

    "03.13.01": "Boundary Protection",
    "03.13.02": "Security Engineering",
    "03.13.03": "Role Separation",
    "03.13.04": "Shared Resource Control",
    "03.13.05": "Public-Access System Separation",
    "03.13.06": "Network Communication by Exception",
    "03.13.07": "Split Tunneling",
    "03.13.08": "Data in Transit",
    "03.13.09": "Connections Termination",
    "03.13.10": "Key Management",

    "03.13.11": "CUI Encryption",
    "03.13.12": "Collaborative Device Control",
    "03.13.13": "Mobile Code",
    "03.13.14": "Voice over Internet Protocol",
    "03.13.15": "Communications Authenticity",
    "03.13.16": "Data at Rest",

    "03.14.01": "Flaw Remediation",
    "03.14.02": "Malicious Code Protection",
    "03.14.03": "Security Alerts & Advisories",
    "03.14.04": "Update Malicious Code Protection",
    "03.14.05": "System & File Scanning",
    "03.14.06": "Monitor Communications for Attacks",
    "03.14.07": "Identify Unauthorized Use"
}


In [24]:
import requests
import os
import json
import re 

# URL to fetch the JSON data
url = "https://csrc.nist.gov/extensions/nudp/services/json/nudp/framework/version/sp_800_171_2_0_0/export/json?element=all"

# File path to store the JSON data
framework_file_path = "../client/public/data/sp_800_171_2_0_0/framework.json"

# Fetch the JSON data
response = requests.get(url)
framework_data = response.json()

# Regex match form 3.14.1 and 03.14
pattern = re.compile(r'\d{1}\.\d{1,2}(?:\.\d{1,2})?')

def normalize(match):
    parts = match.group().split(".")
    return ".".join(f"{int(p):02d}" for p in parts)


# Convert v2 id to v3
for element in framework_data["response"]["elements"]["elements"]:
    identifier = pattern.search(element['element_identifier'])

    if identifier:
        v3_identifier = ".".join(map(lambda c: c.rjust(2, '0'), identifier.group(0).split(".")))
        element['element_identifier'] = element['element_identifier'].replace(identifier.group(0), v3_identifier)

    if element['element_type'] == 'family':
        element['title'] = element['title'].title().replace('And', 'and').replace('Of', 'of')

    if element['element_type'] == 'requirement':
        element['title'] = missing_req_titles[element['element_identifier']]

    if element['element_type'] == 'discussion':
        element['text'] = pattern.sub(normalize, element['text'])

with open('r2.json', 'r') as r2:
    framework_data['response']['elements']['elements'] = framework_data['response']['elements']['elements'] + json.load(r2)


# Ensure the directory exists
os.makedirs(os.path.dirname(framework_file_path), exist_ok=True)

# Write the JSON data to the file
with open(framework_file_path, 'w') as file:
    json.dump(framework_data, file, indent=4)

print(f"JSON data has been stored in {framework_file_path}")

JSON data has been stored in ../client/public/data/sp_800_171_2_0_0/framework.json
